In [ ]:
import json
import numpy as np
import pandas
from datetime import datetime
import time
from pprint import pprint
import re
import warnings
pandas.options.display.max_colwidth = -1

optional_info_list = []#['players']

data_dir = '/home/idies/workspace/Temporary/raddick/baseball_scratch/'

print('imported')

In [ ]:
# GET FULL MLB ROSTER (use later to find handedness)
s = time.time()
full_roster_df = pandas.read_csv('fullroster.csv', low_memory=False, index_col='Unnamed: 0')
full_roster_df.index.name = 'arbitraryIndex'
#full_roster_df[full_roster_df['position'] != 'P'].drop_duplicates('playerID').groupby('playerID')[
#    'batting_hand'].count().sort_values(ascending=False).sort_index() # yay, nobody switched batting hands!
#full_roster_df[full_roster_df['position'] == 'P'].drop_duplicates('playerID').groupby('playerID')[
#    'throwing_hand'].count().sort_values(ascending=False).sort_index() # or pitching hands!
mlb_players_df = full_roster_df.drop_duplicates('playerID')
mlb_players_df = mlb_players_df.set_index('playerID')
mlb_players_df = mlb_players_df.drop('year', axis=1)
players_more_df = pandas.read_csv('players.csv')
players_more_df = players_more_df.set_index('id')
players_more_df.index.name = 'playerID'
mlb_players_df = mlb_players_df.join(players_more_df['debut'])
e = time.time()
print('All-time MLB roster loaded: Found {0:,.0f} players in {1:.2f} seconds!'.format(len(mlb_players_df), e-s))

In [ ]:
def get_starters(thisgame):
# Get starters
    for z in range(0, len(thisgame['visiting_starters'])):
        thisgame['visiting_starters'][z]['which_team'] = 0
        thisgame['visiting_starters'][z]['order'] = int(thisgame['visiting_starters'][z]['order'])
        thisgame['visiting_starters'][z]['position'] = int(thisgame['visiting_starters'][z]['position'])
    for z in range(0, len(allgames[i]['home_starters'])):
        thisgame['home_starters'][z]['which_team'] = 1
        thisgame['home_starters'][z]['order'] = int(thisgame['home_starters'][z]['order'])
        thisgame['home_starters'][z]['position'] = int(thisgame['home_starters'][z]['position'])
    playing_list = allgames[i]['home_starters'] + thisgame['visiting_starters']   
    return playing_list
    
def record_current_players(play_df, playing_list, current_batting_team):
    for m in range(0, len(playing_list)):
        if ((playing_list[m]['which_team'] == 1-current_batting_team) & (playing_list[m]['position'] == 2)):
            play_df = play_df.assign(fieldC = playing_list[m]['player_id'])
        if ((playing_list[m]['which_team'] == 1-current_batting_team) & (playing_list[m]['position'] == 3)):
            play_df = play_df.assign(field1B = playing_list[m]['player_id'])
        if ((playing_list[m]['which_team'] == 1-current_batting_team) & (playing_list[m]['position'] == 4)):
            play_df = play_df.assign(field2B = playing_list[m]['player_id'])
        if ((playing_list[m]['which_team'] == 1-current_batting_team) & (playing_list[m]['position'] == 5)):
            play_df = play_df.assign(field3B = playing_list[m]['player_id'])
        if ((playing_list[m]['which_team'] == 1-current_batting_team) & (playing_list[m]['position'] == 6)):
            play_df = play_df.assign(fieldSS = playing_list[m]['player_id'])
        if ((playing_list[m]['which_team'] == 1-current_batting_team) & (playing_list[m]['position'] == 7)):
            play_df = play_df.assign(fieldLF = playing_list[m]['player_id'])            
        if ((playing_list[m]['which_team'] == 1-current_batting_team) & (playing_list[m]['position'] == 8)):
            play_df = play_df.assign(fieldCF = playing_list[m]['player_id'])            
        if ((playing_list[m]['which_team'] == 1-current_batting_team) & (playing_list[m]['position'] == 9)):
            play_df = play_df.assign(fieldRF = playing_list[m]['player_id'])            
    return play_df


def make_substitution(substitution_event, nowplaying, teamatbat):
# MAKE SUBTITUTION, if this is a sub event 

    substitution_event['sub_team'] = int(substitution_event['sub_team'])
    substitution_event['sub_batting_order'] = int(substitution_event['sub_batting_order'])
    substitution_event['sub_position'] = int(substitution_event['sub_position'])
    # batting sub
    if (substitution_event['sub_team'] == teamatbat):
        if (debug == 3):
            print('BATTING SUB')
            if (substitution_event['sub_position'] == 11):
                print('pinch hitter')                                
            if (substitution_event['sub_position'] == 12):
                print('pinch runner')
        for m in range(0, len(nowplaying)):
            if ((nowplaying[m]['which_team'] == substitution_event['sub_team']) & (nowplaying[m]['order'] == substitution_event['sub_batting_order'])):
                if (debug == 3):
                    print('Originally: {0:} at position {1:.0f} in batting order {2:.0f}'.format(nowplaying[m]['player_id'], nowplaying[m]['position'], nowplaying[m]['order']))
                nowplaying[m]['player_id'] = substitution_event['sub_id']
                nowplaying[m]['order'] = substitution_event['sub_batting_order']
                if (debug == 3):                  
                    print('Substituted: {0:} at position {1:.0f} in batting order {2:.0f}'.format(nowplaying[m]['player_id'], nowplaying[m]['position'], nowplaying[m]['order']))
                    print('\n')
    # fielding sub
    else:
        if (debug == 3):
            print('FILEDING SUB')
        for m in range(0, len(nowplaying)):
            if ((nowplaying[m]['which_team'] == substitution_event['sub_team']) & (nowplaying[m]['position'] == allgames[i]['events'][j]['sub_position'])):
                if (debug == 3):
                    print('Originally: {0:} at position {1:.0f} in batting order {2:.0f}'.format(nowplaying[m]['player_id'], nowplaying[m]['position'], nowplaying[m]['order']))
                nowplaying[m]['player_id'] = substitution_event['sub_id']
                nowplaying[m]['order'] = substitution_event['sub_batting_order']
                nowplaying[m]['position'] = substitution_event['sub_position']
                if (debug == 3):                  
                    print('Substituted: {0:} at position {1:.0f} in batting order {2:.0f}'.format(playing_list[m]['player_id'], nowplaying[m]['position'], nowplaying[m]['order']))
                    print('\n')
    return nowplaying

def init_game():
    current_status_dict = {}
    current_status_dict['gameID'] = allgames[i]['id']
    current_status_dict['playnum'] = 0
    current_status_dict['current_inning'] = 1
    current_status_dict['current_batting_team'] = 0 # 0 = visitor, 1 = home
    current_status_dict['nOuts'] = 0 
    
    onbasedict = {}
    onbasedict['1'] = np.nan
    onbasedict['2'] = np.nan
    onbasedict['3'] = np.nan
    current_status_dict['onbase'] = onbasedict
    
    current_status_dict['score'] = [0,0]  # score as [vis, home] starts at [0,0]
    current_status_dict['batter_id'] = ''
    current_status_dict['batter_hand'] = ''
    current_status_dict['pitcher_id'] = ''
    current_status_dict['pitcher_hand'] = ''
    return current_status_dict

def get_pitcher(status, nowplaying):
    pitcher_id = [
        x['player_id'] for x in nowplaying if (
            x['which_team'] == 1-status['current_batting_team']
        ) & (
            x['position'] == 1)
    ]
    return pitcher_id

def get_pitcher_hand(thispitcher, mlb_players_df):
    pitcher_hand = get_hand(thispitcher, 'p', mlb_players_df)
    return pitcher_hand
    
def get_batter_hand(thisbatter, mlb_players_df, current_pitcher_hand):
    batter_hand = get_hand(thisbatter, 'b', mlb_players_df)
    # if switch-hitter, hit to advantage
    if (batter_hand == 'B'):
        if (current_pitcher_hand == 'L'):
            batter_hand = 'R'
        elif (current_pitcher_hand == 'R'):
            batter_hand = 'L'
        else:
            print('Error in function get_batter_hand: pitcher handnedness not found!')
    
    return batter_hand

def get_hand(thisplayer, batting_or_pitching, mlb_players_df):
    if (batting_or_pitching.lower() == 'p'):
        varname = 'throwing_hand'
    elif (batting_or_pitching.lower() == 'b'):
        varname = 'batting_hand'
    else:
        print('Error in function get_hand(): need to specify batter or pitcher!')
    hand = str(mlb_players_df[varname].loc[thisplayer])
    return hand

def get_play_context(this_play_df, current_status_dict):
    # record the context of this play
    this_play_df = this_play_df.assign(play_number = current_status_dict['playnum'])
    this_play_df = this_play_df.assign(inning = current_status_dict['current_inning'])
    this_play_df = this_play_df.assign(batting_team = current_status_dict['current_batting_team'])
    this_play_df = this_play_df.assign(outs = current_status_dict['nOuts'])

    return this_play_df

def split_event(df):
    df['play'] = df['event'].str.split('/').str.get(0)
    df['play'] = df['play'].str.split('.').str.get(0)
    
    df['modifiers'] = df['event'].str.split('/',n=1).str.get(1)
    df['modifiers'] = df['modifiers'].fillna('')

    df['baserunning'] = df['event'].str.split('.',n=1).str.get(1)
    df['baserunning'] = df['baserunning'].fillna('')
    df['modifiers'] = df['modifiers'].str.split('.').str.get(0)
    
    return df


'''
                  

                this_play_df = this_play_df.assign(batter_hand = mlb_players_df['batting_hand'].loc[this_play_df['batter_id']].get_values()[0])

                
                                    # Get pitcher handedness (and correct if there is a padj)
                    if (re.search('padj: ', thisevent['comment'].lower())):   # correct if there is a badj
                        thisevent['pitcher_hand'] = thisevent['comment'][re.search('padj: ', thisevent['comment'].lower()).end():re.search('padj: ', thisevent['comment'].lower()).end()+1]
                    else:
                        thisevent['pitcher_hand'] = roster_df['throwing_hand'][
                            (roster_df['playerID'] == thisevent['pitcherID']) & 
                            (roster_df['year'] == float(thisevent['gameID'][3:7])) 
                            ].tolist()[0]
                                                             
                    # Get the batter's handnedness
                    if (re.search('badj: ', thisevent['comment'].lower())):   # correct if there is a badj
                        thisevent['batter_hand'] = thisevent['comment'][re.search('badj: ', thisevent['comment'].lower()).end():re.search('badj: ', thisevent['comment'].lower()).end()+1]
                    else:
                        thisevent['batter_hand'] = roster_df['batting_hand'][(roster_df['playerID'] == thisevent['batterID']) & (roster_df['team'] == thisevent['batting_team']) & (roster_df['year'] == float(thisevent['gameID'][3:7]))].tolist()[0]
                    
                    # If a switch hitter, he hits to advantage
                    if (thisevent['batter_hand'] == 'S'):
                        if (thisevent['pitcher_hand'] == 'L'):
                            thisevent['batter_hand'] = 'R'
                        elif (thisevent['pitcher_hand'] == 'R'):
                            thisevent['batter_hand'] = 'R'
                        else:
                            print('Batter handedness not found')

                
                
    
'''    

#                    # First task: parse baserunning information and advance runners accordingly.
#                       # Must do this regardless of the outcome of the plate appearance because of sacrifices
#                    x = thisevent['baserunning'].split(';')
#
#                    success = []
#                    failure = []
#
#                    # Find successful advances by looking for "-" and failed by looking for "X"
#                    for i in range(0, len(x)):
#                        if ('-' in x[i]):
#                            success.append([x[i].split('-')[0], x[i].split('-')[1][0], x[i].split('-')[1][1:]])
#                        if ('X' in x[i]):
#                            failure.append([x[i].split('X')[0], x[i].split('X')[1][0], x[i].split('X')[1][1:]])
#
#                    # Change explicit batter advance (B) to base #0 and scoring play (H) to base #4 for int-ifying later.
#                    for u in (success, failure):
#                        for j in range(0, len(u)):
#                            if (u[j][0] == 'B'):
#                                u[j][0] = 0
#                            if (u[j][1] == 'H'):
#                                u[j][1] = 4
#                            for k in range(0,2):
#                                u[j][k] = int(u[j][k])
#
#                    # For each failure event, increment outs and clear starting base
#                    for y in range(0, len(failure)):
#                        nOuts = nOuts + 1
#                        onbase[failure[y][0]-1] = ''
#                        if (failure[y][1] < 4):            # no need to clear a base if runner is out at home
#                            onbase[failure[y][1]-1] = ''
#                        thisevent['comment'] += failure[y][2]     
#
#                    # For each success event, advance all runners
#                    for z in range(0, len(success)):
#                        if (success[z][1] == 4):
#                            score[side] = score[side] + 1
#                        elif (success[z][0] == 0):
#                            onbase[success[z][1]-1] = thisevent['batterID']
#                        else:
#                            onbase[success[z][1]-1] = onbase[success[z][0]-1]
#
#                        onbase[success[z][0]-1] = ''   # regardless, clear the base he was originally on
#                        thisevent['comment'] += success[z][2]
#
#                        # NEED TO ADD:
#                        #    1) Set GIDP flag if batter has two runners are out (including the batter)
#                        #    2) Un-set batter's ab_flag if this is a sacrifice                        
                        
#                        if (debug == 6):
#                            print('Baserunning info parsed!')
#                            print('Now on base: ',onbase)
#                            print('New score: Visitor {0:.0f} Home {1:.0f}, with {2:.0f} outs.'.format(score[0], score[1], nOuts))
#                            print('\n')

print('Functions defined')


In [ ]:
## 0: no debug output, show final results only
## 1: show file parsing info
## 2: show inning and batting team
## 3: show sub info
## 4: show batter handedness info
## 5: show play outcome
## 6: show baserunning logic

outfinder = re.compile('^[\d|F]+')

debug = 6
global_event_number = 0
plays_df = pandas.DataFrame()

#yeararray = list(range(1921,2018))
yeararray = list(range(2017,2018))
# iterate through years to load data

for thisyear in yeararray:
    print('Year {0:.0f}'.format(thisyear))
    # runs in about 5 minutes
    filename = data_dir + 'events_by_year/events_{0:.0f}.json'.format(thisyear)
    with open(filename,'r') as f:
        allgames = json.load(f)
    
    # iterate through each game in this year
    #for i in range(0, len(allgames)):
    for i in range(929, 933):
        if (debug >= 1):
            if ((np.mod(i,1) == 0) | (i == len(allgames)-1)):
                print('Parsing game {0:,.0f} of {1:,.0f}: {2:}...'.format(i, len(allgames), allgames[i]['id']))
                
        # initialize this game so we can track status through events
        current_status_dict = init_game()        
        playing_list = get_starters(allgames[i])
            
        # iterate through this game's events
        for j in range(0, len(allgames[i]['events'])):            
            # store everything about this event in thisevent_df data frame (index is global event number)
            this_play_df = pandas.DataFrame(data = [global_event_number], columns=['eventID'])
            this_play_df = this_play_df.assign(gameID = allgames[i]['id'])
            this_play_df = this_play_df.assign(visteam = allgames[i]['info']['visteam'])
            
            # advance current inning, if needed
            try:
                allgames[i]['events'][j]['inning'] = int(allgames[i]['events'][j]['inning'])
                if (allgames[i]['events'][j]['inning'] > current_status_dict['current_inning']):
                    current_status_dict['current_inning'] = current_status_dict['current_inning'] + 1
                    current_status_dict['nOuts'] = 0
            except KeyError:
                pass
            # switch current batting team, if needed
            try:
                allgames[i]['events'][j]['visitor_or_home'] = int(allgames[i]['events'][j]['visitor_or_home'])
                if (allgames[i]['events'][j]['visitor_or_home'] != current_status_dict['current_batting_team']):
                    current_status_dict['current_batting_team'] = 1 - current_status_dict['current_batting_team']
                    current_status_dict['nOuts'] = 0
            except KeyError:
                pass
            
            if (debug == 2):
                print('Inning {0:.0f}, team {1:} batting...'.format(current_status_dict['current_inning'], current_status_dict['current_batting_team']))            
            
            if (allgames[i]['events'][j]['event_type'] == 'sub'):
                thisgameid = allgames[i]['id']            
                if (debug == 3):
                    print('Substitution in game {0:}, inning {1:.0f}, batting team: {2:.0f}...'.format(thisgameid, current_status_dict['current_inning'], current_status_dict['current_batting_team']))
                playing_list = make_substitution(allgames[i]['events'][j], playing_list, current_status_dict['current_batting_team'])
            
            elif (allgames[i]['events'][j]['event_type'] == 'play'):
                
                this_play_df = get_play_context(this_play_df, current_status_dict)
                
                # get batter
                this_play_df = this_play_df.assign(batter_id = allgames[i]['events'][j]['batter_id'])
                
                # get pitcher and pitcher's handedness
                this_play_df = this_play_df.assign(pitcher_id = get_pitcher(current_status_dict, playing_list))
                this_play_df = this_play_df.assign(pitcher_hand = get_pitcher_hand(this_play_df['pitcher_id'].get_values()[0], mlb_players_df))
    
                # get batter's handedness (must do after pitcher known because switch hitters)
                this_play_df = this_play_df.assign(batter_hand = get_batter_hand(this_play_df['batter_id'].get_values()[0], mlb_players_df, this_play_df['pitcher_hand'].get_values()[0]))
                                
                #print(mlb_players_df['batting_hand'].loc[this_play_df['batter_id']])
                if ('players' in optional_info_list):
                    this_play_df = record_current_players(this_play_df, playing_list, current_status_dict['current_batting_team'])

                if (debug == 4):
                    print('Play {0:}: pitcher: {1:}, hand: {2:}; batter: {3:}, hand: {4:}'.format(
                        current_status_dict['playnum'],
                        this_play_df['pitcher_id'].get_values()[0],
                        this_play_df['pitcher_hand'].get_values()[0],
                        this_play_df['batter_id'].get_values()[0],
                        this_play_df['batter_hand'].get_values()[0])
                         )
                
                this_play_df = this_play_df.assign(event = allgames[i]['events'][j]['event'])
                # parse event into play, modifiers, baserunning info
                this_play_df = split_event(this_play_df)
                
                # find outcome to be used later in determining consequences
                outcome = this_play_df['event'].get_values()[0]
                
                if (this_play_df['play'].get_values()[0][0:2] == 'NP'):
                    outcome = 'no play'
                if (this_play_df['play'].get_values()[0][0:2] in ['SB', 'CS', 'DI']):
                    outcome = 'stolen base logic'
                elif (this_play_df['play'].get_values()[0] == 'K'):
                    outcome = 'strikeout'
                elif (this_play_df['play'].get_values()[0][0] == 'K'):
                    outcome = 'strikeout plus'
                elif (re.match(outfinder, this_play_df['play'].get_values()[0])):
                    outcome = 'fielder out'
                elif (
                    (this_play_df['play'].get_values()[0] == 'W')
                    | (this_play_df['play'].get_values()[0][0:2] in ['HP', 'WP', 'CI', 'IW'])
                ):
                    outcome = 'batter walk to first'
                elif (this_play_df['play'].get_values()[0][0] == 'E'):
                    outcome = 'error'
                elif (this_play_df['play'].get_values()[0][0] == 'S'):
                    outcome = 'single'
                elif (this_play_df['play'].get_values()[0][0] == 'D'):
                    outcome = 'double'
                elif (this_play_df['play'].get_values()[0][0] == 'T'):
                    outcome = 'triple'
                elif (this_play_df['play'].get_values()[0][0:2] == 'HR'):
                    outcome = 'home run'
                                        
                if (debug >= 5):
                    print('Inning {0:}: batting team {1:}, outs {2:} play {3:}, outcome {4:} on play #{5:}'.format(
                        current_status_dict['current_inning'], 
                        current_status_dict['current_batting_team'],
                        current_status_dict['nOuts'],
                        this_play_df['event'].values[0],
                        outcome,
                        this_play_df['play_number'].values[0])
                     )
                    if (debug == 6):
                        if (outcome not in ['strikeout','strikeout plus','fielder out']):\
                            print('Event requires baserunning info...')
                                                    
                # record the plate appearance events that happened on this play
                this_play_df = this_play_df.assign(balls = allgames[i]['events'][j]['pitch_count'][0])
                this_play_df = this_play_df.assign(strikes = allgames[i]['events'][j]['pitch_count'][1])
                this_play_df = this_play_df.assign(pitch_sequence = allgames[i]['events'][j]['pitch_sequence'])

                # calculate consequences of this play
                
                # if there is baserunning info, implement it first
                if (this_play_df['baserunning'].get_values()[0] != ''):
                    if (debug == 6):
                        print('\n')
                        print('BASERUNNING info found!')
                        print('Baserunning info: ', this_play_df['baserunning'].get_values()[0])


                
                if (this_play_df['play'][0][0] == 'K'):
                    current_status_dict['nOuts'] = current_status_dict['nOuts'] + 1
                elif (re.match(outfinder, allgames[i]['events'][j]['event'][0])):
                    current_status_dict['nOuts'] = current_status_dict['nOuts'] + 1
                #elif (allgames[i]['events'][j]['event'])
                
                # reset outs for new inning
                if (current_status_dict['nOuts'] >= 3):
                    current_status_dict['nOuts'] = 0

                # if it's a no-play, don't add to events df, don't increment play number
                if (allgames[i]['events'][j]['event'] != 'NP'):
                    # Add this event to the df containing all this game's events, unless it's a no-play
                    plays_df = pandas.concat((plays_df, this_play_df), sort=False)
                    # lastly, increment current play number
                    current_status_dict['playnum'] = current_status_dict['playnum'] + 1
            else:
                pprint(allgames[i]['events'][j])
            
            # Lastly, increment global event number
            global_event_number = global_event_number + 1
            
            
plays_df = plays_df.set_index('eventID')
column_order = ['gameID', 'play_number', 'visteam', 'inning', 'batting_team']
column_order +=  ['outs', 'balls', 'strikes', 'pitch_sequence', 'batter_id', 'batter_hand', 'pitcher_id', 'pitcher_hand']
if ('players' in optional_info_list):
    column_order += ['fieldC','field1B','field2B','field3B','fieldSS','fieldLF','fieldCF','fieldRF']
column_order += ['play', 'modifiers', 'baserunning']
plays_df = plays_df[column_order]
#plays_df
print('Done')

In [ ]:
#for j in range(50,77):
#    pprint(allgames[929]['events'][j])
plays_df[['gameID','play_number','play']][(plays_df['gameID'] == 'HOU201705260')]

In [ ]:
# GAMES WITH UNUSUAL EVENTS

# BADJ
#BSN192406190 (1924 GAME 19)
y = [x['player_id'] for x in playing_list if (x['which_team'] == 1) & (x['position'] == 1)][0]
print(y)

In [ ]:
pprint(allgames[i]['events'][j])

In [ ]:


        
#        for j in range(0,14):
            

            # if this event is a play
                #thisgame_df = split_event(thisgame_df)
                pass
            else:
                # TEST FOR BADJ: 1930 game 96
                # TEST FOR PADJ: 1995 game 2,013
                pprint(allgames[i]['events'][j-1])

                pprint(allgames[i]['events'][j])
                
                pprint(allgames[i]['events'][j+1])


print('done')

#    thisgame_df = pandas.DataFrame(thisrow['events'], columns=column_list)
#    thisgame_df = thisgame_df.fillna('')

#all_rosters_df
#thisgame_df

#playing_df

In [ ]:
pprint(allgames[i]['events'][j])

In [ ]:
debug = 1

import json
import numpy as np
import pandas
from datetime import datetime
import time
from pprint import pprint
import re
pandas.options.display.max_colwidth = -1
print('imported')

yeararray = [1995]

#column_list = ['event_in_game', 'event_type', 'inning', 'visitor_or_home', 'batter_id']
#column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_team', 'sub_id', 'sub_batting_order', 'sub_position']
#player_columns = ['player_id', 'order', 'position']

all_rosters_df = pandas.read_csv('fullroster.csv')
all_rosters_df['year'] = pandas.to_numeric(all_rosters_df['year'], errors='coerce')

for thisyear in yeararray:   
    with open('events_json/events_{0:.0f}.json'.format(thisyear,'r')) as f:
        allgames = json.load(f)

# iterate through all games
    for i in range(0, len(allgames)):
#    for i in range(2013,2014):
        # regardless of debug, print a progress message every 50th game
#        if ((np.mod(i,50) == 0) | (i == len(allgames) - 1)):
#            print('Parsing game {0:,.0f}...\n'.format(i))
        if (debug >= 1):
            print('Parsing game {0:,.0f}: {1:}'.format(i, allgames[i]['id']))

        for z in range(0, len(allgames[i]['visiting_starters'])):
            allgames[i]['visiting_starters'][z]['which_team'] = 0
            allgames[i]['visiting_starters'][z]['order'] = int(allgames[i]['visiting_starters'][z]['order'])
            allgames[i]['visiting_starters'][z]['position'] = int(allgames[i]['visiting_starters'][z]['position'])
        for z in range(0, len(allgames[i]['home_starters'])):
            allgames[i]['home_starters'][z]['which_team'] = 1
            allgames[i]['home_starters'][z]['order'] = int(allgames[i]['home_starters'][z]['order'])
            allgames[i]['home_starters'][z]['position'] = int(allgames[i]['home_starters'][z]['position'])
            
        playing_list = allgames[i]['home_starters'] + allgames[i]['visiting_starters']

        score = [0,0] # score as [vis, home] starts at [0,0]
        current_inning = 1
        current_batting_team = 0
        
#        for j in range(0,14):
        for j in range(0, len(allgames[i]['events'])):
#            pprint(allgames[i]['events'][j])
            
            # advance current inning
            try:
                allgames[i]['events'][j]['inning'] = int(allgames[i]['events'][j]['inning'])
                if (allgames[i]['events'][j]['inning'] > current_inning):
                    current_inning = current_inning + 1
            except KeyError:
                pass
            
            # switch current batting team
            try:
                allgames[i]['events'][j]['visitor_or_home'] = int(allgames[i]['events'][j]['visitor_or_home'])
                if (allgames[i]['events'][j]['visitor_or_home'] != current_batting_team):
                    current_batting_team = 1 - current_batting_team
            except KeyError:
                pass
            
            # MAKE SUBTITUTION, if this is a sub event 
            if (allgames[i]['events'][j]['event_type'] == 'sub'):
                allgames[i]['events'][j]['sub_team'] = int(allgames[i]['events'][j]['sub_team'])
                allgames[i]['events'][j]['sub_batting_order'] = int(allgames[i]['events'][j]['sub_batting_order'])
                allgames[i]['events'][j]['sub_position'] = int(allgames[i]['events'][j]['sub_position'])

                # batting sub
                if (allgames[i]['events'][j]['sub_team'] == current_batting_team):
                    if (debug >= 2):
                        print('BATTING SUB')
                        print('Inning: {0:.0f}, batting team: {1:}'.format(current_inning, current_batting_team))
                        if (allgames[i]['events'][j]['sub_position'] == 11):
                            print('pinch hitter')                                
                        if (allgames[i]['events'][j]['sub_position'] == 12):
                            print('pinch runner')
                    if (debug >=2):
                        print('Requested sub: {0:} at position {1:.0f} in batting order {2:.0f}'.format(allgames[i]['events'][j]['sub_id'], allgames[i]['events'][j]['sub_position'], allgames[i]['events'][j]['sub_batting_order']))
                    for m in range(0, len(playing_list)):
                        if ((playing_list[m]['which_team'] == allgames[i]['events'][j]['sub_team']) & (playing_list[m]['order'] == allgames[i]['events'][j]['sub_batting_order'])):
                            if (debug >= 2):
                                print('Originally: {0:} at position {1:.0f} in batting order {2:.0f}'.format(playing_list[m]['player_id'], playing_list[m]['position'], playing_list[m]['order']))
                            playing_list[m]['player_id'] = allgames[i]['events'][j]['sub_id']
                            playing_list[m]['order'] = allgames[i]['events'][j]['sub_batting_order']
                            if (debug >= 2):                  
                                print('Substituted: {0:} at position {1:.0f} in batting order {2:.0f}'.format(playing_list[m]['player_id'], playing_list[m]['position'], playing_list[m]['order']))
                                print('\n')
                # fielding sub
                else:
                    if (debug >= 2):
                        print('FIELDING SUB')
                        print('Inning: {0:.0f}, batting team: {1:}'.format(current_inning, current_batting_team))
                        print('Requested sub: {0:} at position {1:.0f} in batting order {2:.0f}'.format(allgames[i]['events'][j]['sub_id'], allgames[i]['events'][j]['sub_position'], allgames[i]['events'][j]['sub_batting_order']))
                    for m in range(0, len(playing_list)):
                        if ((playing_list[m]['which_team'] == allgames[i]['events'][j]['sub_team']) & (playing_list[m]['position'] == allgames[i]['events'][j]['sub_position'])):
                            if (debug >= 2):
                                print('Originally: {0:} at position {1:.0f} in batting order {2:.0f}'.format(playing_list[m]['player_id'], playing_list[m]['position'], playing_list[m]['order']))
                            playing_list[m]['player_id'] = allgames[i]['events'][j]['sub_id']
                            playing_list[m]['order'] = allgames[i]['events'][j]['sub_batting_order']
                            playing_list[m]['position'] = allgames[i]['events'][j]['sub_position']
                            if (debug >= 2):                  
                                print('Substituted: {0:} at position {1:.0f} in batting order {2:.0f}'.format(playing_list[m]['player_id'], playing_list[m]['position'], playing_list[m]['order']))
                                print('\n')

            # if this event is a play
            elif (allgames[i]['events'][j]['event_type'] == 'play'):
                #thisgame_df = split_event(thisgame_df)
                pass
            else:
                # TEST FOR BADJ: 1930 game 96
                # TEST FOR PADJ: 1995 game 2,013
                pprint(allgames[i]['events'][j-1])

                pprint(allgames[i]['events'][j])
                
                pprint(allgames[i]['events'][j+1])


print('done')

#    thisgame_df = pandas.DataFrame(thisrow['events'], columns=column_list)
#    thisgame_df = thisgame_df.fillna('')

#all_rosters_df
#thisgame_df

#playing_df

In [ ]:
def get_starters(onegame):
    vsdf = pandas.DataFrame(onegame['visiting_starters'], columns=player_columns)
    vsdf['which_team'] = 0
    hsdf = pandas.DataFrame(onegame['home_starters'], columns=player_columns)
    hsdf['which_team'] = 1
    df = pandas.concat((vsdf, hsdf))
    df['order'] = pandas.to_numeric(df['order'])
    df['position'] = pandas.to_numeric(df['position'])       
    df = df.reset_index(drop=True)
#    df = df.set_index('player_id')

    return df

print('functions parsed')

In [ ]:
# runs in about 5 minutes
with open('events_json/events_1921.json','r') as f:
    allgames = json.load(f)
print('ok')
#pprint(allgames[2])   
allgames_bk = allgames

allgames_df = pandas.DataFrame(allgames)
allgames_df = allgames_df.set_index('id')
print('done')
#allgames_df.head(1)

In [ ]:
#pdf: TOR201604250
#bdf: TOR201604250
#ldf: OAK201309040
#allgames_bk_df = allgames_df
'
#mygames = ['SEA201208150']
#somecolumns = allgames_df.columns

#allgames2_df = pandas.DataFrame(columns=somecolumns)
#allgames2_df['id'] = ''
#allgames2_df = allgames2_df.set_index('id')

#for thisid, thisrow in allgames_df.iterrows():
#    if (thisid in mygames):
#        allgames2_df = allgames2_df.append(thisrow)

#allgames_df = allgames2_df

#print('done')
#allgames_df

In [ ]:
# debug values:
## 0: no debug output, show final results only
## 1: show basic information about each event
## 2: also show info about the outcome of a batter's plate appearance
## 3: also show outcome of baserunning for this event
## 4: EVERYTHING

debug = 0

print('debug = {:.0f}'.format(debug))
print('\n')

nGames = len(allgames_df)
cnt = 0

roster_df = load_rosters()

events_df = pandas.DataFrame()

#start = time()

for thisgameID,thisrow in allgames_df.iterrows():
    cnt = cnt + 1
#    if (numpy.mod(cnt,100) == 0):
    print('Processing game {0:,.0f} of {1:,.0f}: {2}...'.format(cnt, nGames, thisgameID))
#    isnow = time()
#    clocker = isnow - start
    game_events_df = pandas.DataFrame()
        
    thisgame_df = pandas.DataFrame(thisrow['events'], columns=column_list)
    thisgame_df = thisgame_df.fillna('')
    
    thisgame_df['gameID'] = thisgameID
    thisgame_df['inning'] = pandas.to_numeric(thisgame_df['inning'])
    thisgame_df['home_or_visitor'] = pandas.to_numeric(thisgame_df['home_or_visitor'])
    thisgame_df['sub_team'] = pandas.to_numeric(thisgame_df['sub_team'], errors='coerce')
    thisgame_df['sub_batting_order'] = pandas.to_numeric(thisgame_df['sub_batting_order'], errors='coerce')
    thisgame_df['sub_position'] = pandas.to_numeric(thisgame_df['sub_position'], errors='coerce')
    
    
    thisgame_df = thisgame_df.set_index('event_in_game')
    
    this_info_df = get_game_info(thisrow, allgames_df.index) # get constants of game info
    playing_df = get_starters(thisrow) # get starters
    thisgame_df = split_event(thisgame_df) # split all events in game into parts    
    score = [0,0] # score as [vis, home] starts at [0,0]

    foundinnings = thisgame_df['inning'].drop_duplicates().values.tolist()
    theseinnings = numpy.arange(1, numpy.max(foundinnings)+1) 

    for ing in theseinnings:
        for side in range(0,2):                           # iter through each half-inning
            if (debug > 0):
                print('Inning {0:.0f}, side {1:.0f}'.format(ing, side))
                print('\n')
            halfinn = thisgame_df[(thisgame_df['inning'] == ing) & (thisgame_df['home_or_visitor'] == side)]
            onbase = ['', '', '']
            nOuts = 0            #     and there are no outs
            nOuts = 0
            fieldingflag = 1 - side  # flag: makes fielding team the opposite of batting team

            for eig, thisevent in halfinn.iterrows():    # iter through all events in this half-inn
#                prev_clock = time.process_time()
                
                if (debug > 0):
                    print('Event number {0:.0f}'.format(eig))
#                    clocker = (time.process_time() - prev_clock) * 1000
#                    print('That took {0:,.3f} milliseconds'.format(clocker))

                if (thisevent['event_type'] == 'sub'):
                    thisyear = this_info_df['date'].apply(lambda x: x.year)
                    (playing_df, onbase) = make_substitution(playing_df, roster_df, thisevent, this_info_df, onbase)
                    
                else:
                    if (debug > 0):
                        print('Batter is {0:s}, outcome is {1:s}'.format(thisevent['batterID'], thisevent['event']))
                        if (thisevent['comment'] != ''):
                            print('Comment found: {:s}'.format(thisevent['comment']))

                    # fill in constants for this play
                    thisevent['visiting_team'] = this_info_df['visteam'].tolist()[0]
                    thisevent['home_team'] = this_info_df['hometeam'].tolist()[0]
                    thisevent['date'] = this_info_df['date'].tolist()[0]
                    thisevent['inning'] = ing
                    if (side == 0):
                        thisevent['batting_team'] = thisevent['visiting_team']
                        thisevent['fielding_team'] = thisevent['home_team']
                    else:
                        thisevent['batting_team'] = thisevent['home_team']
                        thisevent['fielding_team'] = thisevent['visiting_team']

                        
                    # Get the pitcher for this play
                    thisevent['pitcherID'] = playing_df['playerID'][(playing_df['which_team'] == fieldingflag) & (playing_df['position'] == 1)].tolist()[0]                               
                    
                            
                    # Get the fielders for this play                        
                    thisevent['catcherID'] = playing_df['playerID'][(playing_df['which_team'] == fieldingflag) & (playing_df['position'] == 2)].tolist()[0]
                    thisevent['1bID'] = playing_df['playerID'][(playing_df['which_team'] == fieldingflag) & (playing_df['position'] == 3)].tolist()[0]
                    thisevent['2bID'] = playing_df['playerID'][(playing_df['which_team'] == fieldingflag) & (playing_df['position'] == 4)].tolist()[0]
                    thisevent['3bID'] = playing_df['playerID'][(playing_df['which_team'] == fieldingflag) & (playing_df['position'] == 5)].tolist()[0]
                    thisevent['ssID'] = playing_df['playerID'][(playing_df['which_team'] == fieldingflag) & (playing_df['position'] == 6)].tolist()[0]
                    thisevent['lfID'] = playing_df['playerID'][(playing_df['which_team'] == fieldingflag) & (playing_df['position'] == 7)].tolist()[0]
                    thisevent['cfID'] = playing_df['playerID'][(playing_df['which_team'] == fieldingflag) & (playing_df['position'] == 8)].tolist()[0]
                    thisevent['rfID'] = playing_df['playerID'][(playing_df['which_team'] == fieldingflag) & (playing_df['position'] == 9)].tolist()[0]
                        
                                
                    # Get balls and strikes
                    thisevent['balls'] = pandas.to_numeric(thisevent['pitch_count'][0:1], errors='coerce')
                    thisevent['strikes'] = pandas.to_numeric(thisevent['pitch_count'][1:2], errors='coerce')

                    thisevent['abflag'] = 1
                    

                    # get the state of the game at the time this play begins
                    thisevent['outs'] = nOuts
                    thisevent['runner1b'] = onbase[0]
                    thisevent['runner2b'] = onbase[1]
                    thisevent['runner3b'] = onbase[2]
                    thisevent['visitor_score'] = score[0]
                    thisevent['home_score'] = score[1]


                    # if there is baserunning info, implement it first
                    if (thisevent['baserunning'] != ''):
                        
                        if (debug >= 3):
                            print('\n')
                            print('BASERUNNING info found!')
                            print('Baserunning info: ', thisevent['baserunning'])

                        # First task: parse baserunning information and advance runners accordingly.
                           # Must do this regardless of the outcome of the plate appearance because of sacrifices
                        x = thisevent['baserunning'].split(';')

                        success = []
                        failure = []

                        # Find successful advances by looking for "-" and failed by looking for "X"
                        for i in range(0, len(x)):
                            if ('-' in x[i]):
                                success.append([x[i].split('-')[0], x[i].split('-')[1][0], x[i].split('-')[1][1:]])
                            if ('X' in x[i]):
                                failure.append([x[i].split('X')[0], x[i].split('X')[1][0], x[i].split('X')[1][1:]])

                        # Change explicit batter advance (B) to base #0 and scoring play (H) to base #4 for int-ifying later.
                        for u in (success, failure):
                            for j in range(0, len(u)):
                                if (u[j][0] == 'B'):
                                    u[j][0] = 0
                                if (u[j][1] == 'H'):
                                    u[j][1] = 4
                                for k in range(0,2):
                                    u[j][k] = int(u[j][k])

                        # For each failure event, increment outs and clear starting base
                        for y in range(0, len(failure)):
                            nOuts = nOuts + 1
                            onbase[failure[y][0]-1] = ''
                            if (failure[y][1] < 4):            # no need to clear a base if runner is out at home
                                onbase[failure[y][1]-1] = ''
                            thisevent['comment'] += failure[y][2]     

                        # For each success event, advance all runners
                        for z in range(0, len(success)):
                            if (success[z][1] == 4):
                                score[side] = score[side] + 1
                            elif (success[z][0] == 0):
                                onbase[success[z][1]-1] = thisevent['batterID']
                            else:
                                onbase[success[z][1]-1] = onbase[success[z][0]-1]

                            onbase[success[z][0]-1] = ''   # regardless, clear the base he was originally on
                            thisevent['comment'] += success[z][2]

                        # NEED TO ADD:
                        #    1) Set GIDP flag if batter has two runners are out (including the batter)
                        #    2) Un-set batter's ab_flag if this is a sacrifice                        
                        
                        if (debug >= 3):
                            print('Baserunning info parsed!')
                            print('Now on base: ',onbase)
                            print('New score: Visitor {0:.0f} Home {1:.0f}, with {2:.0f} outs.'.format(score[0], score[1], nOuts))
                            print('\n')

                        # PARSE THE RESULT OF THE BATTER'S PLATE APPEARANCE
                    if (debug >= 2):
                        print('Entering plate appearance on play number {:,.0f}'.format(eig))
                        
                    while True:
                        if (re.match(r'[0-9K]+', thisevent['play'])):
                            nOuts = nOuts + 1
                            if (debug >= 2):
                                print('out')
                                print('\n')
                            break
                            
                        # if no play, it's not an at-bat
                        elif (thisevent['play'][0] == 'N'):
                            if (debug >= 2):
                                print('no play')
                            thisevent['abflag'] = 0
                            break
                            
                        # baserunning events not involving the batter
                        elif (re.match(r'WP|PB|SB|CS|DI|PO|BK', thisevent['play'])):
                            thisevent['abflag'] = 0   # not an at-bat for any of these events

                            if (debug >= 2):
                                print('baserunning event not involving batter')
                                if (re.match(r'WP', thisevent['play'])):
                                    print('Wild pitch')
                                if (re.match(r'PB', thisevent['play'])):
                                    print('Passed ball')
                                if (re.match(r'SB', thisevent['play'])):
                                    print('Stolen base')
                                if (re.match(r'CS', thisevent['play'])):
                                    print('Caught stealing')
                                if (re.match(r'DI', thisevent['play'])):
                                    print('Defensive indifference')
                                if (re.match(r'PO', thisevent['play'])):
                                    print('Pickoff')
                                if (re.match(r'BK', thisevent['play'])):
                                    print('Balk')
                                
                            # If defensive interference, advance has already been recorded in baserunning, so just break.
                            if (re.match(r'DI|BK', thisevent['play'])):
                                break
                            
                            # If stolen base, advance base-stealer
                            if (re.match(r'SB', thisevent['play'])):
                                
                                # parse the base that got stolen
                                stolen_base = thisevent['play'][2]
                                # Again, set scoring play (stealing home) to 4 
                                if (stolen_base == 'H'):
                                    stolen_base = 4
                                else:
                                    stolen_base = int(stolen_base)
                                
                                # get base started from
                                # THIS WON'T WORK FOR DOUBLE STEALS!
                                orig_base = stolen_base - 1            
                                if (stolen_base < 4):   # only move ahead one base if home not stolen
                                    onbase[stolen_base-1] = onbase[orig_base-1]
                                onbase[orig_base-1] = ''
                                break

                            # If caught stealing, clear the base the runner started from and increment outs
                            if (re.match(r'CS', thisevent['play'])):          
                                # parse the base at which they were caught stealing
                                caught_base = thisevent['play'][2]

                                # Again, set scoring play (stealing home) to 4
                                if (caught_base == 'H'):
                                    caught_base = 4
                                else:
                                    caught_base = int(caught_base)
                                if (debug >= 3):
                                    print('Caught stealing at base {0:.0f}'.format(caught_base))                                    
                                orig_base = caught_base - 1
                                nOuts = nOuts + 1
                                onbase[orig_base-1] = ''
                                break    # don't forget to break out of loop no matter what
                            
                            if (re.match(r'PO', thisevent['play'])):
                                # parse the base at which the player was picked off
                                try:
                                    picked_base = int(thisevent['play'][2])
                                except:
                                    picked_base = 3
                                onbase[picked_base-1] = ''
                                nOuts = nOuts + 1
                                if (debug >= 2):
                                    print('Picked off!!!!!!!!!!!!!!')
                                    print('Now on base: ',onbase)
                                    print('Now {0:.0f} outs'.format(nOuts))
                                break
                            
                        # single/FC/error: batter to first with at-bat
                        elif ((thisevent['play'][0] in ('S', 'E')) or (re.match(r'FC', thisevent['play']))):
                            if (debug >= 2):
                                if (thisevent['play'][0] == 'S'):
                                    print('Single')
                                if (thisevent['play'][0] == 'E'):
                                    print('Error')
                                if (re.match(r'FC', thisevent['play'])):
                                    print('Fielders choice')
                            
                            # regardless of what happened, check whether batter advance given explicitly
                            if ('B' not in thisevent['baserunning']):
                                onbase[0] = thisevent['batterID']    # if not, advance batter to first

                            if (debug >= 3):
                                print('Now on base: ',onbase)
                            break    # don't forget to break out of loop no matter what
                            
                        # double
                        elif (thisevent['play'][0] == 'D'):
                            if (debug >= 2):
                                print('Double')
                            # as above, move batter (to 2nd) unless specificied otherwise
                            if ('B' not in thisevent['baserunning']):
                                onbase[1] = thisevent['batterID']
                            if (debug >= 3):
                                print('Now on base: ',onbase)
                            break    # don't forget to break out of loop no matter what
                        
                        # triple
                        elif (thisevent['play'][0] == 'T'):
                            if (debug >= 2):
                                print('Triple')
                            # as above, move batter (to 3rd) unless specified above
                            if ('B' not in thisevent['baserunning']):
                                onbase[2] = thisevent['batterID']
                            if (debug >= 3):
                                print('Now on base: ',onbase)
                            break    # don't forget to break out of loop no matter what

                        # home run
                        elif (re.match(r'HR', thisevent['play'])):
                            if (debug >= 2):
                                print('Home run')
                            # add 1 to score since batter crossed home plate
                            score[side] = score[side] + 1
                            # clear bases
                            onbase = ['', '', '']
                            break # don't forget to break out of loop no matter what
                            
                        # Walk or catcher interference...
                        # ...we already know it's not a WP or CS or HR because it would have been caught above
                        elif (thisevent['play'][0] in ('W', 'C', 'H', 'I', 'F', 'O')):
                            if (debug >= 2):
                                if (thisevent['play'][0] == 'W'):
                                    print('Walk')
                                if (thisevent['play'][0] == 'C'):
                                    print('Catcher interference')
                                if (thisevent['play'][0] == 'H'):
                                    print('Hit by pitch')
                                if (thisevent['play'][0] == 'I'):
                                    print('Intentional walk')
                                if (thisevent['play'][0] == 'F'):
                                    print('Fly ball error')
                                
                                # advance batter to 1st unless otherwise specified
                                if ('B' not in thisevent['baserunning']):
                                    onbase[0] = thisevent['batterID']
                                break # don't forget to break out of loop no matter what
                        
                        # if not found yet, print a message saying so.
                        else:
                            print('not found yet!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                            print('Play: ',thisevent['play'])
                        break
                            
                        
 
                # finally, append this play to the list of plays
                game_events_df = game_events_df.append(thisevent)
            if (debug > 0):
                print('\n')
                
    game_events_df = game_events_df[game_events_df['event_type'] == 'play']
    game_events_df = game_events_df[game_events_df['play'] != 'NP']

    game_events_df = game_events_df.reset_index()
    game_events_df.index.name = 'event_in_game'
    
    events_df = pandas.concat((events_df, game_events_df), axis=0)

#events_df = events_df.fillna('')

events_df = events_df.reset_index()

events_df['event_in_game'] = events_df['event_in_game'] + 1  # make it a 1-based counter
events_df['date'] = pandas.to_datetime(events_df['date'], errors='coerce')

events_df.index.name = 'eventID'

events_df.sort_values('gameID')

#display_order = ['gameID', 
display_order = ['event_in_game', 'inning', 'home_or_visitor', 'outs']
#display_order += ['batterID', 'batter_hand', 'pitcherID', 'pitcher_hand']
display_order += ['batterID', 'play', 'modifiers', 'baserunning', 'comment']
display_order += ['runner1b', 'runner2b', 'runner3b']
display_order += ['visitor_score', 'home_score']

#print('Writing file...')
events_df.to_csv('events/events_2016.csv', encoding='utf-8')
print('Done!')

#print('Final score: ',score)
#events_df = events_df[display_order]
#events_df
#events_df.groupby('outs').size()

In [ ]:
column_list = ['event_in_game', 'event_type', 'inning', 'home_or_visitor', 'batterID']
column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_team', 'sub_batting_order', 'sub_position']

player_columns = ['playerID', 'name', 'batting_order', 'position']

display_order = ['gameID', 'date', 'event_in_game', 'visiting_team', 'home_team', 'inning', 'home_or_visitor']
display_order += ['visitor_score', 'home_score', 'batterID', 'batter_hand', 'pitcherID', 'pitcher_hand', 'abflag']
display_order += ['catcherID', '1bID', '2bID', '3bID', 'ssID', 'lfID', 'cfID', 'rfID']
display_order += ['outs', 'balls', 'strikes', 'pitch_sequence']
display_order += ['runner1b', 'runner2b', 'runner3b']
display_order += ['play', 'modifiers', 'baserunning', 'comment']
print('got columns')

def load_rosters():
    all_rosters_df = pandas.read_csv('fullroster.csv')
    all_rosters_df['year'] = pandas.to_numeric(all_rosters_df['year'], errors='coerce')
    return all_rosters_df

def get_game_info(onerow, theindex):

    info_df = pandas.DataFrame.from_records(onerow['info'], index=theindex)

    teams_df = pandas.read_csv('teams.csv')
    info_df['visleague'] = teams_df['league'][teams_df['teamID'] == info_df['visteam'].tolist()[0]].tolist()[0][0:1]
    info_df['homeleague'] = teams_df['league'][teams_df['teamID'] == info_df['hometeam'].tolist()[0]].tolist()[0][0:1]
#    try:
#        info_df['attendance'] = pandas.to_numeric(info_df['attendance'], errors='coerce')
#    except:
#        info_df['attendance'] = -1
    info_df['date'] = pandas.to_datetime(info_df['date'], errors='coerce')
#    info_df['dayofweek'] = info_df['date'].apply(lambda x: datetime.strftime(x, '%a'))
    
    info_df['gameno'] = pandas.to_numeric(info_df['number'], errors='coerce')
#    info_df = info_df.drop('number', axis=1)
#    info_df['starttime'] = pandas.to_datetime(info_df['starttime'], format='%l%M%P', errors='coerce')
#    info_df['temp'] = pandas.to_numeric(info_df['temp'], errors='coerce')
#    try:
#        info_df['timeofgame'] = pandas.to_numeric(info_df['timeofgame'], errors='coerce')
#    except:
#        info_df['timeofgame'] = -1
    info_df = info_df.fillna('')

    return info_df


def get_starters(onerow):
    vsdf = pandas.DataFrame(onerow['visiting_starters'], columns=player_columns)
    vsdf['which_team'] = 0
    hsdf = pandas.DataFrame(onerow['home_starters'], columns=player_columns)
    hsdf['which_team'] = 1
    df = pandas.concat((vsdf, hsdf))
    df['batting_order'] = pandas.to_numeric(df['batting_order'])
    df['position'] = pandas.to_numeric(df['position'])

    return df


def split_event(df):
    df['play'] = df['event'].str.split('/').str.get(0)
    df['play'] = df['play'].str.split('.').str.get(0)
    
    df['modifiers'] = df['event'].str.split('/',n=1).str.get(1)
    df['modifiers'] = df['modifiers'].fillna('')

    df['baserunning'] = df['event'].str.split('.',n=1).str.get(1)
    df['baserunning'] = df['baserunning'].fillna('')
    df['modifiers'] = df['modifiers'].str.split('.').str.get(0)
    
    return df


def get_sub_name(players_curr, theroster, evnt, gameinfo):
    
    if (evnt['sub_team'] == 0):
        sub_team_name = gameinfo['visteam'].values[0]
    else:
        sub_team_name = gameinfo['hometeam'].values[0]
    yeer = gameinfo['date'].apply(lambda x: x.year).values[0]
    try:
        sub_name_first = theroster['nameFirst'].iloc[
                numpy.where(
                    (theroster['playerID'] == evnt['batterID']) & 
                    (theroster['year'] == yeer) &
                    (theroster['team'] == sub_team_name)
                )
            ].values[0]
    except:
        sub_name_first = ''
    try:
        sub_name_last = theroster['nameLast'].iloc[
                numpy.where(
                    (theroster['playerID'] == evnt['batterID']) & 
                    (theroster['year'] == yeer) &
                    (theroster['team'] == sub_team_name)
                )
            ].values[0]
    except:
        sub_name_last = ''
    sub_name = '"' + sub_name_first + ' ' + sub_name_last + '"'

    return sub_name

def make_substitution(players_curr, theroster, evnt, gameinfo, basers):
#    print('\n')
#    print('SUBSTITUTING...')
    if (evnt['home_or_visitor'] == evnt['sub_team']):
        
        if (evnt['sub_position'] == 12):
#            print('pinch runner')
            old_runner = players_curr['playerID'].loc[
                (players_curr['which_team'] == evnt['sub_team']) & 
                (players_curr['batting_order'] == evnt['sub_batting_order'])
            ].values[0]
            new_runner = evnt['batterID']
#            print(old_runner, new_runner)
            for i in range(0,len(basers)):
                if (basers[i] == old_runner):
                    basers[i] = new_runner
#            print(basers)
        
        players_curr.loc[
            (players_curr['which_team'] == evnt['sub_team']) & 
            (players_curr['batting_order'] == evnt['sub_batting_order'])
            , 'playerID'] = evnt['batterID']

        players_curr.loc[
            (
                (players_curr['which_team'] == evnt['sub_team']) & 
                (players_curr['batting_order'] == evnt['sub_batting_order'])
                ), 'name'] = get_sub_name(players_curr, theroster, evnt, gameinfo)
        
    else:
        
        players_curr.loc[
            (
                (players_curr['which_team'] == evnt['sub_team']) & 
                (players_curr['position'] == evnt['sub_position'])
                ), 'playerID'] = evnt['batterID']
        
        players_curr.loc[
            (
                (players_curr['which_team'] == evnt['sub_team']) & 
                (players_curr['position'] == evnt['sub_position'])
                ), 'name'] = get_sub_name(players_curr, theroster, evnt, gameinfo)
#        print('New lineup:\n')
#    print(players_curr)
    return (players_curr, basers)

print('Done')

In [ ]:
allgames_bk_df = allgames_df
print('Done')

In [ ]:
allgames_df = allgames_bk_df
#allgames_df = allgames_df[15:17]
#allgames_df = allgames_df.loc['BSN192705070']
print('Done')

In [ ]:
allgames_df['info'].sample(1).sort_values()

In [ ]:
current_status_dict